Import libraries

In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

데이터 로드

In [ ]:
df=pd.read_csv(#####빈칸을 채우세요######)
df

데이터 전처리

In [ ]:
date_time = pd.to_datetime(df.pop('time_x'), format='%Y-%m-%d %H:%M:%S.%f')
df=df.drop(['gear_fault_desc'], axis=1)
df = df[[#####빈칸을 채우세요######]]

In [ ]:
df.head()

In [ ]:
df.describe()

Train, test 데이터 분할

In [ ]:
#데이터 확인
plot_cols = ['speedSet', 'load_value','sensor1', 'sensor2']
plot_features = df[plot_cols]
_ = plot_features.plot(subplots=True)

In [ ]:
#train/test data 분할
# 7개 데이터가 입력으로 들어가고 batch size는 임의로 지정
seq_length = 7
batch = 100

data=0
train_set = #####빈칸을 채우세요######
test_set = #####빈칸을 채우세요######


In [ ]:
train_set

In [ ]:
test_set

In [ ]:
plot_cols = ['speedSet', 'load_value','sensor1','sensor2']
plot_features = train_set[plot_cols]
_ = plot_features.plot(subplots=True)

plot_features = test_set[plot_cols]
_ = plot_features.plot(subplots=True)

데이터 스케일링

In [ ]:
# Input scale
train_set = train_set.copy()
test_set = test_set.copy()

scaler_x = MinMaxScaler()
scaler_x.fit(train_set.iloc[:, :-1])

train_set.iloc[:, :-1] = scaler_x.transform(#####빈칸을 채우세요######)
test_set.iloc[:, :-1] = scaler_x.transform(#####빈칸을 채우세요######)

# Output scale
scaler_y = MinMaxScaler()
scaler_y.fit(train_set.iloc[:, [-1]])


train_set.iloc[:, -1] = scaler_y.transform(#####빈칸을 채우세요######)
test_set.iloc[:, -1] = scaler_y.transform(#####빈칸을 채우세요######)

In [ ]:
train_set.head()

데이터셋 생성

In [ ]:
device = torch.device('cpu')
# 데이터셋 생성 함수
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series)-seq_length):
        _x = time_series[#####빈칸을 채우세요######]
        _y = time_series[#####빈칸을 채우세요######]
        # print(_x, "-->",_y)
        dataX.append(_x)
        dataY.append(_y)

    return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(np.array(train_set), seq_length)
testX, testY = build_dataset(np.array(test_set), seq_length)


# 텐서로 변환
trainX_tensor = torch.FloatTensor(trainX)
trainY_tensor = torch.FloatTensor(trainY)

testX_tensor = torch.FloatTensor(testX)
testY_tensor = torch.FloatTensor(testY)

testX_tensor = testX_tensor.to(device)
testY_tensor = testY_tensor.to(device)
# 텐서 형태로 데이터 정의
dataset = TensorDataset(trainX_tensor, trainY_tensor)
# 데이터로더는 기본적으로 2개의 인자를 입력받으며 배치크기는 통상적으로 2의 배수를 사용
dataloader = DataLoader(dataset,
                        batch_size=batch,
                        shuffle=True,
                        drop_last=True)

In [ ]:
train_set[0:10]

LSTM

In [ ]:
import torch.nn as nn

class Net(nn.Module):
    # # 기본변수, layer를 초기화해주는 생성자
    def __init__(self, input_dim, hidden_dim, seq_len, output_dim, layers):
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.seq_len = seq_len
        self.output_dim = output_dim
        self.layers = layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=layers,
                            #dropout = 0.1,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim, bias = True)

    # 학습 초기화를 위한 함수
    def reset_hidden_state(self):
        self.hidden = (
                torch.zeros(self.layers, self.seq_len, self.hidden_dim),
                torch.zeros(self.layers, self.seq_len, self.hidden_dim))

    # 예측을 위한 함수
    def forward(self, x):
        x, _status = self.lstm(x)
        x = self.fc(x[:, -1])
        return x

Training

In [ ]:
def train_model(model, train_df, num_epochs = None, lr = None, verbose = 10, patience = 10):

    criterion = nn.MSELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    nb_epochs = num_epochs

    # epoch마다 loss 저장
    train_hist = np.zeros(nb_epochs)

    for epoch in range(nb_epochs):
        avg_cost = 0
        total_batch = len(train_df)

        for batch_idx, samples in enumerate(train_df):

            x_train, y_train = samples

            # seq별 hidden state reset
            model.reset_hidden_state()

            # H(x) 계산
            outputs = model(x_train)

            # cost 계산
            loss = criterion(outputs, y_train)

            # cost로 H(x) 개선
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            avg_cost += loss/total_batch

        train_hist[epoch] = avg_cost

        if epoch % verbose == 0:
            print('Epoch:', '%04d' % (epoch), 'train loss :', '{:.4f}'.format(avg_cost))

        # patience번째 마다 early stopping 여부 확인
        if (epoch % patience == 0) & (epoch != 0):

            # loss가 커졌다면 early stop
            if train_hist[epoch-patience] < train_hist[epoch]:
                print('\n Early Stopping : %04d epoch' %(epoch))

                break

    return model.eval(), train_hist

In [ ]:
print(trainX_tensor[0:2])
print(trainY_tensor[0:2])

학습 시작

In [ ]:
# 모델 학습
# 설정값
data_dim = #####빈칸을 채우세요######
hidden_dim = #####빈칸을 채우세요######
output_dim = #####빈칸을 채우세요######
learning_rate = #####빈칸을 채우세요######
n_layer=#####빈칸을 채우세요######
nb_epochs = #####빈칸을 채우세요######
net = Net(data_dim, hidden_dim, seq_length, output_dim, n_layer)
model, train_hist = train_model(net, dataloader, num_epochs = nb_epochs, lr = learning_rate, verbose = 10, patience = 10)

In [ ]:
# 
fig = plt.figure(figsize=(10, 4))
plt.plot(train_hist, label="Training loss")
plt.legend()
plt.show()

In [ ]:
# 모델 저장
PATH =#####빈칸을 채우세요######
#torch.save(model.state_dict(), PATH)

# 불러오기
model = Net(data_dim, hidden_dim, seq_length, output_dim, n_layer)
model.load_state_dict(torch.load(PATH), strict=False)
model.eval()
model.to(device)

모델 테스트

In [ ]:
# 예측테스트
a=0
testX_tensor_100=testX_tensor[#####빈칸을 채우세요######]
testY_tensor_100=testY_tensor[#####빈칸을 채우세요######]
testX_tensor_100 = testX_tensor_100.to(device)
testY_tensor_100 = testY_tensor_100.to(device)

with torch.no_grad():
    pred = []
    for pr in range(len(#####빈칸을 채우세요######)):

        model.reset_hidden_state()

        predicted = model(torch.unsqueeze(#####빈칸을 채우세요######[pr], 0))
        predicted = torch.flatten(predicted).item()
        pred.append(predicted)

    # INVERSE

    pred_np = np.array(pred).reshape(-1, 1)
    pred_inverse = scaler_y.inverse_transform(pred_np)

    
    testY_np = testY_tensor_100.cpu().numpy()
    testY_inverse = scaler_y.inverse_transform(#####빈칸을 채우세요######)


In [ ]:
fig = plt.figure(figsize=(15,3))
plt.plot(np.arange(len(pred_inverse)), pred_inverse, label = 'pred')
plt.plot(np.arange(len(testY_inverse)), testY_inverse, label = 'true')
plt.title("Test plot")
plt.legend()
plt.show()